In [1]:
import xarray as xr
import iris
import iris.plot as iplt
import matplotlib.pyplot as plt
import cloudpickle
import pandas as pd
import time
from eofs.iris import Eof
from eofs.multivariate.iris import MultivariateEof
import datetime
import numpy as np
# import netcdf4 as nc4

/home/vis/anaconda3/envs/klima/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/vis/anaconda3/envs/klima/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# Preprocessing

1. __Ausschneiden der geographischen Region__:<br>
    Relevante geographische Region:(𝝺 = [10 °W, 25 °E],𝞅 = [32.5 °N, 67.5 °N)<br>
    Dieser Schritt wurde bereits beim Datendownload berücksichtigt.
    
    
2. __Berechnen & Normieren der Anomalien__:<br>
    Diese Aufgabe wurde in der folgenden Funktion _preprocessing_ ausgeführt.<br>


In [2]:
def preprocessing(data):
    # Berechnen des Mittelwerts
    mean=data.rolling(time=21,center=True).mean().dropna('time')
    climatology=mean.groupby("time.dayofyear").mean('time')
    
    # Berechnen der Standardabweichung 
    data_for_std=data.rolling(time=21,center=True).construct("rolling_days")
    std_dayofyear=data_for_std.groupby("time.dayofyear").std(dim=xr.ALL_DIMS)
    
    # Berechnen der Anomalien
    diff=data.groupby("time.dayofyear")-climatology
    anomalies=diff.groupby("time.dayofyear")/std_dayofyear
    return anomalies

In [3]:
# data=xr.open_dataset("./small_daily_data.nc")

In [4]:
# # Berechnen der Anomalien:
# anomalies=preprocessing(data)

# # Speichern der Anomalien:
# data = Dataset("/home/vis/anomalies_1979-2018.nc4", 'w', format='NETCDF4')
# data = anomalies
# data.close()

In [5]:
# Öffnen der gespeicherten Daten:
#path=/home/vis/
path="/mnt/c/Users/Kathi/Documents/Studium/Master/Klima/Übung/Projekt/"
anomalies=xr.open_dataset(path+"anom_all_1979-2018.nc",chunks={'time': 1, 'lat':141, 'lon': 141})
print(anomalies)

FileNotFoundError: [Errno 2] No such file or directory: b'/mnt/c/Users/Kathi/Documents/Studium/Master/Klima/\xc3\x9cbung/Projekt/anom_all_1979-2018.nc'

In [ ]:
# Testplot der Anomalien
plt.plot(anomalies.r.sel(time=slice("1979-01-01","1981-01-01"),lat=47.0,lon=12.0))

# Analog Methode

## Warum?
Der Skillfull Scale eines globalen Modells, insbesondere eines globalen Klimamodells, liegt bei ca. 800 - 1000 km. Dadurch kann man nicht einfach z.B. globale Klimaprognosen mit regionale Auswirkungen gleichsetzen. <br>
Dazu wird ein Downscaling benötigt, wobei man dabei zwei Arten unterscheidet:
1. Dynamisches Downscaling
2. Statistisches Downscaling

Bei Ersterem wird in das globale Modell ein regionales genestet, was sehr rechenintensiv ist. Bei statistischem Downscaling geht es darum, mit statistischen Hilfsmitteln, Transferfunktionen zu finden, die das globale Klimamodell auf die regionale Skala transformieren, siehe folgende Abbildung:

<img src="lasclosc.png" alt="Statistical Downscaling" style="width: 600px;" />

## Was ist die Analog - Methode?

Um den Transfer von globaler auf lokale Skala zu vollziehen, wir hier die Empirical Orthogonal Function (EOF) - Analyse benützt.<br>
Die Idee hinter der EOF-Analyse ist es, ein Feld durch eine wesentlich geringer Menge an Daten wie das Original auszudrücken, ohne dabei große Einbußen in der Information zu erzeugen. Dabei werden sowohl räumliche (EOF) und zeitliche (Prinicipal Component - PC) Muster aus den Daten bestimmt. <br>

Für die spätere Validierung werden hier alle Tage aller Jahre berechnet. Für jeden Tag werden einige (hier 3) Analog-Tage gesucht. Da z.B. für den 15. Jänner nicht ein Tag im Juli der perfekte Analog-Tag sein wird, wird für die EOF-Analyse jeweils nur ein Zeitfenster von $\pm$10 Tagen um den gesuchten Tag ausgewählt, für alle verfügbaren Jahre. Auf die resultierenden EOFs wird das Feld des gesuchten Tages projeziert wodurch man Pseudo-PCs (PPC) erhält. <br>

Um den ähnlichsten Tag für das Analogon zu finden wird die Norm zwischen den aus der EOF-Analyse gewonnen PCs und den Pseudo-PCS gebildet, nach der Formel:

\begin{equation}
\text{Norm}=\sum{(\text{PPC}-\text{PC})^2}
\end{equation}

Bei der Normbildung muss natürlich aus dem Pool der aktuelle Tag herausgenommen werden, da dieser ja gleich zum Targetday ist.
Um eine größere Auswahl zu haben wird nicht nur der Tag mit der geringsten Norm ausgewählt, sondern die kleinsten x-Tage. x kann beliebig gewählt werden und wird in diesem Projekt mit 3 angenommen.

In [ ]:
Die Idee hinter einer EOF - Analyse ist, den selben Informationsgehalt mit weniger Daten auszudrücken.# Pfad des Speicherortes der Ausgabedaten
#path = "/home/vis/"
path="/mnt/c/Users/Kathi/Documents/Studium/Master/Klima/Übung/Projekt/"

#  Beispieljahr erstellen (kein Schaltjahr)
exampleyear = pd.date_range('1999-01-01', '1999-12-31')

# Wie viele Analoga sollen gesucht werden:
nr = 3

# Länge der Zeitreihe
years = len(anomalies.groupby('time.year'))

In [ ]:
print('Analoga: ',nr)
print('Jahre: ',years)

In [ ]:
# Ausgabelisten erstellen
out_pc = []
out_r = []
out_q = []
out_msl = []

# Ausgabe Array für die Analoga erstellen:
out_analoga = np.zeros((len(anomalies.time), nr+1), dtype='datetime64[s]')
# Der ersten Spalte werden die Zeitelement zugeordnet -> Tage
out_analoga[:,0] = anomalies.time


# Über alle DOY iterieren:
for i, day in enumerate(exampleyear):
    
    # Fenster erstellen: 10 Tage vor und nach dem aktuellen + alle doy auswählen
    window = pd.date_range(day - datetime.timedelta(days=10), day + datetime.timedelta(days=10))
    doy_window = window.dayofyear

    # Alle Daten welche sich im Fenster befinden auswählen und für die EOF-Analyse time attributes = 'T' setzen
    data = anomalies.sel(time = anomalies.time.dt.dayofyear.isin(doy_window))
    data.coords['time'].attrs['axis'] = 'T'
    
    # Daten des aktuellen Tag auswählen -> für die PPC-Berechnung
    data_doy = anomalies.sel(time = anomalies.time.dt.dayofyear.isin(day.dayofyear))

    # Für die Analyse ist es notwendig die Daten in das Iris Format zu wandeln:
    data_iris = iris.cube.CubeList([data.r.to_iris(), data.q.to_iris(), data.msl.to_iris()]).merge() 
    
    # EOF-Analyse starten:
    solver = MultivariateEof(data_iris, weights=None)
    
    # PC berechnen, bis 90% der Varianz erklärt sind
    j = 0
    while(True):
        variance = solver.varianceFraction(neigs=j)
        if(np.sum(variance.data) > 0.9):
            break
        j += 1
    
    # Gefunden PCs und EOFs auswählen:
    pc = solver.pcs(npcs=j)
    eof = solver.eofs(neofs=j)
    
    # Rücktransformation nach fertiger EOF-Analyse und Fertige Berechnung zur Liste hinzufügen -> PC für die Normberechnung später noch notwenig! 
    calculated_pc = xr.DataArray.from_iris(pc).rename('PC')
    out_pc.append(calculated_pc)
    out_r.append(xr.DataArray.from_iris(eof[0]).rename('EOF_r'))
    out_q.append(xr.DataArray.from_iris(eof[1]).rename('EOF_q'))
    out_msl.append(xr.DataArray.from_iris(eof[2]).rename('EOF_msl'))
   
    # Für die Analyse ist es auch hier wieder notwendig die Daten des aktuellen DOY in das Iris Format zu wandeln:
    data_doy_iris = iris.cube.CubeList([data_doy.r.to_iris(), data_doy.q.to_iris(), data_doy.msl.to_iris()]).merge()    

    # Berechnen der PPCs:
    ppc = solver.projectField(data_doy_iris, neofs=j)

    # Rücktransformation:
    data_ppc = xr.DataArray.from_iris(ppc).rename('PPC')

    # Über alle Jahre iterieren:
    for k in range(0, years):
        
        # Berechnen der Norm
        norm = (( calculated_pc - data_ppc[k] )**2).sum(dim='pc')
        
        # Das akutelle Jahr aus dem Pool herausnehmen
        norm = norm.sel(time = ~norm.time.dt.year.isin(data_ppc.time.to_series()[k].year))
        kday = data_ppc.time.to_series().dt.strftime('%Y-%m-%d')[k]
        
        # Reihe im Ausgabe Array festlegen
        row = np.where(out_analoga[:,0] == datetime.datetime.strptime(kday, '%Y-%m-%d'))[0][0]

        #Suche nach dem Analogon für den aktuellen Tag (Anzahl oben festgelegt -> nr)
        for l in range(1,nr+1):
            
            # Norm minimieren -> Analogen finden: kleinste Norm = beste Übereinstimmung
            mini = np.argmin(norm).values
            
            # Das k.te Jahr auslblenden und Analogon bestimmen
            analogon = data.sel(time = ~data.time.dt.year.isin(data_ppc.time.to_series()[k].year)).time.to_series()[mini]
            
            # Analogon im Array ablegen
            out_analoga[row][l] = analogon
            
            # Das bereits gefunden Minimum herausnehmen, um die weiteren finden zu können:
            norm = norm.where(norm.values != norm[mini].values)

    print(day)

In [ ]:
# Speichern der Ausgangsdaten:
cloudpickle.dump( out_r, open( path + "/EOFr_" + analogyear + ".p", "wb" ) )
cloudpickle.dump( out_q, open( path + "/EOFq_" + analogyear + ".p", "wb" ) )
cloudpickle.dump( out_msl, open( path + "/EOFmsl_" + analogyear + ".p", "wb" ) )
cloudpickle.dump( out_analoga, open( path + "/Analoga_" + analogyear + ".p", "wb" ) )
cloudpickle.dump( out_pc, open( path + "/PC_" + analogyear + ".p", "wb" ) )

Die meiste Arbeit war das Verstehen der Aufgabenstellung und das Implementieren dieser. Auch die Konvertierung in Iris hat einige Fragen aufgeworfen, war aber durch Cf - Konformität der ERA5-Daten nich so problematisch wie bei anderen Grupppen.